## USING GFF TO CONVERT V5.3 TO V5.6

In [31]:
import gffpandas.gffpandas as gffpd
import pandas as pd
import numpy as np

#### ISOLATING THE GENE IN GFF v5.3 ####
v5_3 = gffpd.read_gff3('/research/projects/chlamydomonas/MAexpression/analysis/flo/flo/chlamy.5.3.w_organelles_mtMinus_transcripts.gff')
v5_3 = v5_3.df
v5_3_mRNA = v5_3.loc[v5_3['type'] == 'mRNA']
v5_3_mRNA['attributes'].replace('geneName=', 'gene_name=', regex = True, inplace = True)
v5_3_mRNA['attributes'] = v5_3_mRNA['attributes'].str.split(';')
v5_3_mRNA['Name'] = 'nan'

for i in list(v5_3_mRNA.index.values):
    for a in v5_3_mRNA.at[i, 'attributes']:
        if 'Name=' in a:
            v5_3_mRNA.at[i, 'Name'] = a
v5_3_mRNA.replace('nan', np.nan, inplace = True)
v5_3_mRNA.dropna(axis = 0, subset = ['Name'], inplace = True)
v5_3_mRNA['Name'].replace('Name=', '', regex = True, inplace = True)

#### ISOLATING THE GENE IN GFF v5.6 ####
v5_6 = gffpd.read_gff3('/research/projects/chlamydomonas/MAexpression/analysis/flo/flo/Creinhardtii_281_v5.6.exons.gff3')
v5_6 = v5_6.df
v5_6_mRNA = v5_6.loc[v5_6['type'] == 'mRNA']
v5_6_mRNA['attributes'].replace('geneName=', 'gene_name=', regex = True, inplace = True)
v5_6_mRNA['attributes'] = v5_6_mRNA['attributes'].str.split(';')
v5_6_mRNA['Name'] = 'nan'

for i in list(v5_6_mRNA.index.values):
    for a in v5_6_mRNA.at[i, 'attributes']:
        if 'Name=' in a:
            v5_6_mRNA.at[i, 'Name'] = a
v5_6_mRNA.replace('nan', np.nan, inplace = True)
v5_6_mRNA.dropna(axis = 0, subset = ['Name'], inplace = True)
v5_6_mRNA['Name'].replace('Name=', '', regex = True, inplace = True)

for i in list(v5_6_mRNA.index.values):
    ### 90 percentile of gene length in v5.6 ###
    v5_6_length = v5_6_mRNA.at[i, 'end'] - v5_6_mRNA.at[i, 'start']
    percentile = v5_6_length * 0.9
    ### identifying the corresponding sections of v5.3 in v6 ###
    section = v5_3_mRNA.loc[v5_3_mRNA['seq_id'] == v5_6_mRNA.at[i, 'seq_id']]
    section = section.loc[section['strand'] == v5_6_mRNA.at[i, 'strand']]
    for a in list(section.index.values):
        if v5_6_mRNA.at[i, 'start'] <= section.at[a, 'start'] and section.at[a, 'start'] <= v5_6_mRNA.at[i, 'end'] and v5_6_mRNA.at[i, 'start'] <= section.at[a, 'end'] and section.at[a, 'end'] <= v5_6_mRNA.at[i, 'end']:
            v5_6_mRNA.at[i, 'v5.3'] = section.at[a, 'Name']
        if v5_6_mRNA.at[i, 'start'] <= section.at[a, 'start'] and (v5_6_mRNA.at[i, 'end'] - section.at[a, 'start']) >= percentile:
            v5_6_mRNA.at[i, 'v5.3'] = section.at[a, 'Name']
        if v5_6_mRNA.at[i, 'end'] >= section.at[a, 'end'] and (section.at[a, 'end'] - v5_6_mRNA.at[i, 'start']) >= percentile:
            v5_6_mRNA.at[i, 'v5.3'] = section.at[a, 'Name']

v5_6_mRNA['v5_6_gene'] = v5_6_mRNA['Name'].replace('.t[1-9].*','', regex = True)
v5_6_mRNA.to_csv('/research/projects/chlamydomonas/MAexpression/data/genome_info/v5.3_to_v5.6.csv', sep = '\t', index = False, header = True)

/home/balogu15/.conda/envs/personal/lib/python3.8/site-packages/pandas/core/series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-31-b2e2b0779124>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  v5_3_mRNA['attributes'] = v5_3_mRNA['attributes'].str.split(';')
<ipython-input-31-b2e2b0779124>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

0

## USING GFF TO CONVERT V5.6 TO V6

In [89]:
import pandas as pd
import numpy as np
conversion = pd.read_csv('/research/projects/chlamydomonas/MAexpression/data/genome_info/v5_to_v6_liftover/ChlamydomonasTranscriptNameConversionBetweenReleases.Mch12b.csv', delimiter = ',', skiprows = 1)
conversion.columns = ['5.5', '3', '1', 'Genbank', '4', '4.3', 'u5', 'u9', '5.3.1']
conversion = conversion[~conversion.index.duplicated(keep = 'first')]
conversion.drop(['3', '1', 'Genbank', '4', '4.3', 'u5', 'u9'], axis = 1, inplace = True)
conversion = conversion.replace(' ', '', regex = True)
conversion.set_index('5.3.1', inplace = True)
divergence = pd.read_csv('/research/projects/chlamydomonas/MAexpression/data/genome_info/divergence_info/divergence.out.txt', delimiter = '\t', index_col = 'transcript_ID')
PACid2geneID = pd.read_csv('/research/references/chlamydomonas/5.3_chlamy_w_organelles_mt_minus/annotation/PACid2geneID.txt', delimiter = '\t')
PACid2geneID.columns = ['transcript_ID', 'geneid']
PACid2geneID.set_index('transcript_ID', inplace = True)
divergence = divergence.join(PACid2geneID)
divergence.rename(columns = {'geneid': 'v5.3.1'}, inplace = True)
divergence.set_index('v5.3.1', inplace = True)
divergence = divergence.join(conversion)
geneID = pd.read_csv('/research/projects/chlamydomonas/MAexpression/data/genome_info/v5_to_v6_liftover/preliminary_v6_Cre_liftover.tsv', delimiter = '\t', header = None)
geneID.columns = ['v6', 'v5']
geneID.set_index('v5', inplace = True)
geneID = geneID[~geneID.index.duplicated(keep = 'first')]
divergence = divergence.reset_index()
divergence['v5_5_gene'] = divergence['5.5'].replace('.t[1-9].*','', regex = True)
divergence.set_index('v5_5_gene', inplace = True)
divergence = divergence[~divergence.index.duplicated(keep = 'first')]
divergence = pd.concat([divergence, geneID], axis = 1)
divergence = divergence.reset_index()
divergence.rename(columns = {'level_0':'v5_5_gene', 'index':'v5.3.1', '5.5':'JGI_v5.5_ID', 'v6':'JGI_v6'}, inplace = True)
divergence.to_csv('/research/projects/chlamydomonas/MAexpression/analysis/divergence/files/divergence_file_v5.3_and_v6.csv', sep = '\t', header = True, index = True)
chlamy_v5_conversion = divergence[['v5_5_gene', 'v5.3.1', 'JGI_v5.5_ID', 'JGI_v6']]
chlamy_v5_conversion.to_csv('/research/projects/chlamydomonas/MAexpression/data/genome_info/v5.3_to_v5.6_to_v6.csv', sep = '\t', header = True, index = False)